Ideas:
train hugging face model on debate corpus

tag <--> concatted underlined portion of a paragraph in the card

In [9]:
from Vector_Utils import *
import re

In [10]:
prompt = "CO2 destroys crop production"
document = Document("article.docx")
full_text = "\n".join([para.text for para in document.paragraphs])

In [11]:
prompt = llmd_prompt(prompt, full_text)

In [12]:
print("Prompt:", prompt)
emphasis_words = prompt.split()

Prompt: Rising CO2 levels cause extreme weather, damaging crops, and threatening global food security.


In [13]:
vectorizer = vectorizer()
prompt_vec = vectorizer.encode(prompt)

In [14]:
paragraphs = document.paragraphs
sentence_enders = re.compile(r'(?<=[.!?])\s+')
para_split = 2

for paragraph in paragraphs:
    highest = 0.4
    best = ()

    sentences = re.split(sentence_enders, paragraph.text)

    if not vectorizer.screen_paragraph(sentences, prompt_vec, threshold=0.4):
        make_small(paragraph)
        continue

    if len(sentences) > para_split:
        print("SPLIT")
        for i in range((len(sentences) // para_split) + 1):
            k = i * para_split
            print(k)
            if k + para_split <= len(sentences):
                sub_paragraph = " ".join(sentences[k:k+para_split])
            else:
                sub_paragraph = " ".join(sentences[k:])
            print(sub_paragraph)
            # if not vectorizer.screen_paragraph(sub_paragraph, prompt_vec, threshold=0.4):
            #     continue
            sub_highest = 0.4
            sub_best = ()
            clauses = clause_separator(sub_paragraph)
            for combination in greedy_combinations(clauses, round(len(clauses) * 0.8), vectorizer, prompt_vec):
                joined = " ".join(combination)
                vector = vectorizer.encode(joined)
                sim_score = vectorizer.similarity(vector, prompt_vec)
                if sim_score > sub_highest:
                    sub_highest = sim_score
                    sub_best = combination
            best += sub_best
            print(best)
    else:
        clauses = clause_separator(paragraph.text)
        for combination in greedy_combinations(clauses, round(len(clauses) * 0.8), vectorizer, prompt_vec):
            joined = " ".join(combination)
            vector = vectorizer.encode(joined)
            sim_score = vectorizer.similarity(vector, prompt_vec)
            if sim_score > highest:
                highest = sim_score
                best = combination
    
    emphasis = [clause for clause in best if vectorizer.similarity(vectorizer.encode(clause), prompt_vec) > 0.6]
    print(highest, best)
    underline_best_match_in_paragraph(paragraph, best, emphasis)

document.save("article_annotated.docx")

0.7000138759613037
SPLIT
0
Impact of CO2-Caused Climate Changes on Food Production This section looks at the implications of the previous chapters for the planet’s food supply. Extreme Weather (in general) John Podestra, who was President Clinton’s final chief of staff and later became co-chair of President Obama’s transition team, wrote in 2010: “ Food shortages resulting from severe crop losses will occur more frequently and take longer to recover from as more people become vulnerable to extreme weather events”6 In 2011, Oxfam published a report entitled “Extreme Weather Endangers Food Security: 2010-11: A Grim Foretaste of Future Suffering and Hunger?”Romm judged this report, according to which extreme weather had pushed millions of people into hunger, the “climate story of the year,” Oxfam’s report, Romm said, “shows how several extreme weather events have contributed to food insecurity at global, regional and local levels since 2010.”7 In 2012, George Monbiot entitled a story, “If